In [ ]:
!pip install flask diffusers transformers torch accelerate > /dev/null 2>&1
!pip install pyngrok > /dev/null 2>&1
!pip install fastapi uvicorn pyngrok > /dev/null 2>&1
!pip install python-multipart > /dev/null 2>&1
!pip install nest_asyncio > /dev/null 2>&1
!pip install -U -q "google-generativeai>=0.7.2"




In [ ]:
!pip install httpx==0.27.2 > /dev/null 2>&1

This Code is For

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nest_asyncio
nest_asyncio.apply()
import os
from pyngrok import ngrok

# Add your ngrok authentication token
!ngrok authtoken 2pUNMdn5heZDqr50ndLJ9hFA348_7BNec91SNvazozmDtzkf5

# Set the environment variable
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/encoded-blend-439905-g7-3deac22ca1bf.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/service_account_gcp/encoded-blend-439905-g7-3deac22ca1bf.json"



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


This Is Integrating with Fast API

In [ ]:
!pip install pinecone-client > /dev/null 2>&1
!pip install tiktoken > /dev/null 2>&1
!pip install --upgrade pinecone-client > /dev/null 2>&1
!pip install -U langchain-community > /dev/null 2>&1
!pip install -U langchain-openai > /dev/null 2>&1



Final Code to be not changed



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

...

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

KeyboardInterrupt: 

This is the Code to , integrate gemini to enrich the prompt

Good finally working codee

In [ ]:
import shutil

def clear_output_dir(directory):
    """Clears all files in the specified directory."""
    if os.path.exists(directory):
        shutil.rmtree(directory)  # Removes the directory and its contents
        os.makedirs(directory)

In [ ]:
def refine_prompt_with_gemini(user_prompt,gender):
    """Refines the user prompt using Gemini."""
    prompt = f"""
    Refine the user input to describe a unique, stylish dress with specific details for Stable Diffusion model; keep it under 20 words:
    User input: "{user_prompt}" for the gender : {gender}
    """
    model = genai.GenerativeModel('gemini-1.5-flash')
    response = model.generate_content(prompt)
    return response.text.strip()

In [ ]:
from fastapi import FastAPI, Form
from fastapi.middleware.cors import CORSMiddleware
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from langchain_openai.embeddings import OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
from google.cloud import storage
import torch
import uuid
import json
from PIL import Image, ImageDraw, ImageFont
import os
import google.generativeai as genai

# Configure the Generative AI API with your API key
GOOGLE_API_KEY = "GOOGLE_API_KEY"  # Replace with your actual API Key
genai.configure(api_key=GOOGLE_API_KEY)

# Initialize FastAPI app
app = FastAPI()

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Initialize Google Cloud Storage
GCS_BUCKET_NAME = "fashion_designer"
storage_client = storage.Client()
bucket = storage_client.bucket(GCS_BUCKET_NAME)

def upload_to_gcs(file_path, blob_name):
    """Uploads a file to GCS and returns the public URL."""
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(file_path)
    return blob.public_url

# Directory to save temporary output images
OUTPUT_DIR = "./output"
GCP_DIR="./gcp_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(GCP_DIR, exist_ok=True)


# Load Stable Diffusion model
model_path = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")
pipe.enable_attention_slicing()
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Initialize Pinecone with two indexes
pinecone_client = Pinecone(api_key="PINE_CONE_API")
male_index_name = "fashion"
female_index_name = "fashionfemale"

# Create both indexes if they don't exist
for index_name in [male_index_name, female_index_name]:
    if index_name not in [idx.name for idx in pinecone_client.list_indexes()]:
        pinecone_client.create_index(
            name=index_name,
            dimension=1536,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )

# Load the indexes
male_index = pinecone_client.Index(male_index_name)
female_index = pinecone_client.Index(female_index_name)

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings(api_key="OPEN_AI_APIKEY")

@app.post("/generate-image")
async def generate_image(prompt: str = Form(...), gender: str = Form(...)):
    """Handle image generation based on the user input."""
    try:
        # Select the appropriate Pinecone index based on gender
        selected_index = male_index if gender == "Male" else female_index

        # Customize the prompt based on gender
        full_prompt = refine_prompt_with_gemini(prompt,gender)

        # Generate embeddings and query Pinecone
        prompt_embedding = embeddings.embed_query(prompt)
        results = selected_index.query(vector=prompt_embedding, top_k=1, include_metadata=True)

        if results["matches"] and results["matches"][0]["score"] > 0.9:
            # Retrieve cached images from Pinecone
            cached_images = json.loads(results["matches"][0]["metadata"]["images"])
            return {"images": cached_images}

        # Generate new images using Stable Diffusion
        num_variants = 5
        images = pipe([full_prompt] * num_variants, num_inference_steps=25, height=384, width=384).images

        # Upload images to GCP and store URLs
        image_urls = []
        for img in images:
            filename = f"{uuid.uuid4()}.png"
            local_path = os.path.join(OUTPUT_DIR, filename)
            img.save(local_path)
            IMAGE_PROMPT = (
                "Provide only the following details about the images and nothing else"
                " 1. A description in 4-5 words:"
                " 2. The style this dress reflects:"
                " 3. Complementary top and bottom wear color combinations:"
                " Do not include extra text or explanations."
            )

            # Open the image with PIL
            image = Image.open(local_path)

            # Pass image and prompt to Gemini
            model = genai.GenerativeModel('gemini-1.5-flash')
            response = model.generate_content([IMAGE_PROMPT, image])
            caption = response.text.strip()


            # Create output image with caption
            original_image = Image.open(local_path)
            width, height = original_image.size

            # Create a new image with space for the caption
            caption_height = 50  # Space for caption
            new_height = height + caption_height
            new_image = Image.new("RGB", (width, new_height), "white")
            new_image.paste(original_image, (0, 0))

            # Add the caption
            draw = ImageDraw.Draw(new_image)
            font = ImageFont.load_default()  # Customize font if desired
            text_position = (10, height + 10)
            draw.text(text_position, caption, fill="black", font=font)

            # Save the updated image
            captioned_file = f"captioned_{filename}"
            gcp_path = os.path.join(GCP_DIR, captioned_file)
            new_image.save(gcp_path)



            gcs_url = upload_to_gcs(gcp_path, captioned_file)
            image_urls.append(gcs_url)

        # Cache the new prompt and image URLs in the selected Pinecone index
        records = [{
            "id": str(uuid.uuid4()),
            "values": prompt_embedding,
            "metadata": {"prompt": prompt, "images": json.dumps(image_urls)}
        }]
        selected_index.upsert(vectors=records)

        # Return generated images
        return {"images": image_urls}
    except Exception as e:
        return {"error": str(e)}

# Run FastAPI with ngrok
from pyngrok import ngrok
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

import uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

INFO:     Started server process [770]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://2f83-35-197-14-254.ngrok-free.app" -> "http://localhost:8000"
INFO:     131.96.220.154:0 - "POST /generate-image HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [770]
